In [ ]:
import os
import functools

import pandas as pd
import sastvd as svd
import sastvd.linevd.run as lvdrun
import sastvd.helpers.joern as svdj
from ray import tune
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Manager, Pool, Queue

USE_CPU = cpu_count()

In [1]:
def process_joern_parallel(joern_input, queue: Queue):
    func_str, dataset, iid = joern_input
    svdj.full_run_joern_from_string(func_str, dataset, iid)
    return iid

savedir = svd.get_dir(svd.cache_dir() / "minimal_datasets")
df = pd.read_parquet(
                savedir / f"minimal_bigvul_False.pq", engine="fastparquet"
            ).dropna()

funcs = df["before"].tolist()
datasets = df["dataset"].tolist()
iids = df["id"].tolist()

func_cnt = len(funcs)

joern_inputs = [(func_str, dataset, iid) for func_str, dataset, iid in zip(funcs, datasets, iids)]

with Manager() as m:
    message_queue = m.Queue()
    pool = Pool(USE_CPU)
    
    process_func = functools.partial(process_joern_parallel, queue=message_queue)

    done_iids = [
            iid
            for iid in tqdm(
                pool.imap_unordered(process_func, joern_inputs),
                desc=f"Functions",
                total=func_cnt,
            )
        ]

    message_queue.put("finished")
    pool.close()
    pool.join()

NameError: name 'Queue' is not defined

In [10]:
%writefile test_hyperparameter_generation.py
import os

import sastvd as svd
import sastvd.linevd.run as lvdrun
from ray import tune

os.environ["SLURM_JOB_NAME"] = "bash"

config = {
    "hfeat": tune.choice([512]),
    "embtype": tune.choice(["codebert"]),
    "stmtweight": tune.choice([1, 5, 10]),
    "hdropout": tune.choice([0.25, 0.3]),
    "gatdropout": tune.choice([0.15, 0.2]),
    "modeltype": tune.choice(["gat2layer"]),
    "gnntype": tune.choice(["gat"]),
    "loss": tune.choice(["ce"]),
    "scea": tune.choice([0.4, 0.5, 0.6]),
    "gtype": tune.choice(["pdg-raw"]),
    "batch_size": tune.choice([1024]),
    "multitask": tune.choice(["linemethod"]),
    "splits": tune.choice(
        [
            "crossproject_Chrome",
            "crossproject_linux",
            "crossproject_Android",
            "crossproject_ImageMagick",
            "crossproject_php-src",
            "crossproject_tcpdump",
            "crossproject_openssl",
            "crossproject_krb5",
            "crossproject_php",
            "crossproject_qemu",
        ]
    ),
    "lr": tune.choice([1e-3, 1e-4, 3e-4, 5e-4]),
}

if __name__ == "__main__":
    samplesz = -1
    run_id = svd.get_run_id()
    sp = svd.get_dir(svd.processed_dir() / f"raytune_crossproject_{samplesz}" / run_id)
    trainable = tune.with_parameters(lvdrun.train_linevd, samplesz=samplesz, savepath=sp)

    analysis = tune.run(
        trainable,
        resources_per_trial={"cpu": 4, "gpu": 0},
        metric="val_loss",
        mode="min",
        config=config,
        num_samples=1000,
        name="tune_linevd",
        local_dir=sp,
        keep_checkpoints_num=2,
        checkpoint_score_attr="min-val_loss",
    )

2023-03-15 19:12:41,858	WARNING tune.py:146 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
(pid=182804) /home/satyaki/.local/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
(pid=182804)   rank_zero_deprecation(
2023-03-15 19:12:44,671	ERROR tune.py:794 -- Trials did not complete: [train_linevd_0795f_00000, train_linevd_0795f_00001, train_linevd_0795f_00002, train_linevd_0795f_00003, train_linevd_0795f_00004, train_linevd_0795f_00005, train_linevd_0795f_00006, train_linevd_0795f_00007, train_linevd_0795f_00008, train_linevd_0795f_00009, train_linevd_0795f_00010, train_linevd_0795f_00011, train_linevd_0795f_